In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.interpolate import griddata
from numpy.random import random_sample
from Bkspline import Bkspline
from gridvec import gridvec
from griddataXB import griddataXB

plt.rcParams['figure.figsize'] = 10, 8
plt.rcParams.update({'font.size': 9})

# Python adaptation of run_Bkspline.m by Carl Tape
# Coding by Amanda McPherson, 2021

# This is a plotting test function for Bkspline.m, which returns a
# spherical spline basis function at specifited lat-lon points.
# It also returns the spatial derivatives of the basis function,
# which are useful for representating derivatives of target functions,
# as well as for damping.

In [ ]:
# Set up constants, and create sample data

deg = 180/np.pi

# create sample data
numx = 200
ax1 = [-122, -114, 32, 37]
lonmin = ax1[0] 
lonmax = ax1[1]
latmin = ax1[2]
latmax = ax1[3]
lon, lat, numy = gridvec(lonmin,lonmax,numx,latmin,latmax)

# select sample spline (DEFAULT: pick one at random)
q = 6      # KEY: determines the scalelength of the spline (q = 0-10)
clon = (lonmax - lonmin)*random_sample() + lonmin
clat = (latmax - latmin)*random_sample() + latmin

# evaluate spline function
ff = Bkspline(clon, clat, q, lon, lat)

In [ ]:
# Plot sample data

X, Y, Z = griddataXB(lon,lat,ff[:,0],100,'cubic')
plt.pcolor(X,Y,Z,shading='nearest',vmin=0,vmax=1)
plt.axis(ax1)
plt.colorbar()
plt.xlabel('Longitude (deg)')
plt.ylabel('Latitude (deg)')
plt.title('Spherical spline basis function, order q=%i, centered at lon=%.2f, lat=%.2f'% (q,clon,clat))
plt.show()

In [ ]:
# magnitude of surface gradient of spline

ncol = 5
ff = Bkspline(clon, clat, q, lon, lat, ncol=ncol)

dfdp = ff[:,1]
dfdt = ff[:,2]
th   = (90-lat)/deg

# check the computation return Bkspline
dfmag = np.sqrt( dfdt**2 + ((1/np.sin(th).flatten()) * dfdp)**2 )
print(np.linalg.norm( dfmag - ff[:,4] ))

d1max = np.amax([ np.amax(abs(dfdp)), np.amax(abs(dfdt)) ])

# plotting
stitd = ['f','d f / d φ','d f / d θ','∇^2 f','| ∇ f | ']
plt.figure(num=2)

for ii in range(len(stitd)):
    X, Y, Z = griddataXB(lon,lat,ff[:,ii],100,'cubic')
    plt.subplot(3,2,ii+1)
    if ii == 1 or ii == 2:
        plt.pcolor(X,Y,Z,shading='nearest',vmin=-d1max,vmax=d1max)
    else:
        plt.pcolor(X,Y,Z,shading='nearest')
        
    plt.title(stitd[ii])
    plt.axis(ax1)
    plt.colorbar()
    
plt.subplots_adjust(hspace=0.3)    
plt.show()

In [ ]:
# plot the surface gradient
plt.figure(num=3)

# create sample data
lon, lat, numy = gridvec(lonmin,lonmax,20,latmin,latmax)
ff = Bkspline(clon, clat, q, lon, lat, ncol=ncol)

X, Y, Z = griddataXB(lon,lat,ff[:,4],100,'cubic')

plt.pcolor(X,Y,Z,shading='nearest')
plt.quiver(lon,lat,ff[:,1],-ff[:,2],color='k')    # minus sign is to plot the EAST component
plt.title('surface gradient vector field, along with the magnitude')
plt.axis(ax1)
plt.colorbar()

plt.show()